In [143]:
import string

In [144]:
stopwords = nltk.corpus.stopwords.words('english')

In [145]:
ps = nltk.PorterStemmer()

In [146]:
data = pd.read_csv('data/SMSSpamCollection.csv')
data = data[['label','text']]
data.head()
data['text'] = data['text'].apply(str)
# data.dropna(axis=0, inplace=True)


In [147]:
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return count

In [148]:


data['body_len'] = data['text'].apply(lambda x: len(x) - x.count(" "))


In [149]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5574 entries, 0 to 5573
Data columns (total 3 columns):
label       5574 non-null int64
text        5574 non-null object
body_len    5574 non-null int64
dtypes: int64(2), object(1)
memory usage: 130.8+ KB


In [150]:

data['punct%'] = data['text'].apply(lambda x: round(count_punct(x) / len(x), 3) * 100)

In [151]:
data.head()

,label,text,body_len,punct%
0,0,Go until jurong point,18,0.0
1,0,Ok lar... Joking wif u oni...,24,20.7
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,123,0.7
3,0,U dun say so early hor... U c already then say...,39,12.2
4,0,Nah I dont think he goes to usf,24,0.0


In [152]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens =  re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

In [153]:
# vectorizing
from sklearn.feature_extraction.text import TfidfVectorizer

In [154]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text)

In [155]:
X_tfidf = tfidf_vect.fit_transform(data['text'])

In [156]:
data['punct%'].shape

(5574,)

In [157]:
X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_features.shape

(5574, 6870)

In [158]:
# Ready for Gradient Boosting

In [159]:
from sklearn.ensemble import GradientBoostingClassifier

In [160]:
print(dir(GradientBoostingClassifier))

['_SUPPORTED_LOSS', '__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_check_initialized', '_check_params', '_clear_state', '_decision_function', '_estimator_type', '_fit_stage', '_fit_stages', '_get_param_names', '_init_decision_function', '_init_state', '_is_initialized', '_make_estimator', '_resize_state', '_staged_decision_function', '_validate_estimator', '_validate_y', 'apply', 'decision_function', 'feature_importances_', 'fit', 'get_params', 'n_features', 'predict', 'predict_log_proba', 'predict_proba', 'score', 'set_params', 'staged_decision_function', 'staged_predict', 'staged_predic

In [161]:
print(GradientBoostingClassifier())

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)


In [162]:
# GridSearch
from sklearn.model_selection import GridSearchCV


In [168]:
param = {
    'n_estimators': [100, 150],
    'max_depth' : [7, 11, 15],
    'learning_rate' : [0.1]
}

In [164]:
from sklearn.model_selection import train_test_split

In [165]:
X_train, X_test , y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [166]:
X_features.dropna(axis=0, inplace=True)

In [167]:
X_features.shape

(5574, 6870)

In [169]:
gb = GradientBoostingClassifier()
gs = GridSearchCV(gb, param, cv=5, n_jobs=-1)
cv_fit = gs.fit(X_features, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

/home/ghaff/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ghaff/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ghaff/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ghaff/anaconda3/lib/python3.

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
1,196.075104,3.075747,0.208158,0.015662,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.956093,0.965950,0.962298,0.951526,0.959605,0.959096,0.004976,1,0.998878,0.999551,0.999327,0.999327,0.999552,0.999327,2.457625e-04
3,304.020367,5.501340,0.195792,0.005441,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.953405,0.969534,0.961400,0.950628,0.957810,0.958558,0.006613,2,0.999551,0.999776,0.999552,0.999552,0.999776,0.999641,1.098508e-04
2,225.452789,2.816142,0.211943,0.031235,0.1,11,100,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.950717,0.969534,0.959605,0.953321,0.956014,0.957840,0.006550,3,0.999551,0.999551,0.999552,0.999552,0.999552,0.999551,9.855773e-08
0,139.028500,3.942971,0.225100,0.029600,0.1,7,100,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.954301,0.965054,0.961400,0.953321,0.954219,0.957661,0.004704,4,0.995065,0.995289,0.997534,0.993498,0.994395,0.995156,1.342286e-03
4,284.948302,3.814668,0.192097,0.003095,0.1,15,100,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.949821,0.964158,0.961400,0.956014,0.952424,0.956764,0.005364,5,0.999551,0.999776,0.999552,0.999552,0.999776,0.999641,1.098508e-04
